In [ ]:
import os
from functools import partial

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
from wand.image import Image as WImage
import ipywidgets as w

from utils import messages as ms
import ipyvuetify as v
from scripts import export

In [ ]:
#mutable object 
class Export_io:
    def __init__(self):
        self.vue = None
        
export_io = Export_io()

In [ ]:
#result tile 
title="Export results"
id_ = 'export_widget'

export_results = sw.Tile(id_, title, inputs=[''])

In [ ]:
#export data
export_output = sw.Alert().add_msg(ms.EXPORT_INTRO)
export_btn = sw.Btn(ms.EXPORT_BTN, icon='mdi-check')

title = "Export data"

export_txt = sw.Markdown(ms.EXPORT_TXT)

export_tile = sw.Tile(
    id_, 
    title, 
    btn=export_btn, 
    output=export_output,
    inputs=[export_txt]
)

In [ ]:
#export data 
def start_process(widget, event, data, output, result):
    
    #toggle the loading button
    widget.toggle_loading()
     
    #load the input
    file = getattr(tb_io, 'file')
    pts = getattr(tb_io, 'pts')
    bands = getattr(viz_io, 'bands')
    sources = getattr(viz_io, 'sources')
    
    #check only validation     
    if not output.check_input(file, ms.MISSING_IN): return widget.toggle_loading()
    if not output.check_input(bands, ms.MISSING_IN): return widget.toggle_loading()
    if not output.check_input(sources, ms.MISSING_IN): return widget.toggle_loading()
    
    #start the exporting process 
    pdf_file = export.run(file, pts, bands, sources, output)
    
    #add a download button 
    dwn = sw.DownloadBtn('PDF time series', path=pdf_file)
    
    #add a preview of the first page
    preview = pdf_file.replace('.pdf', '_preview.png')
    WImage(filename=pdf_file+'[0]').save(filename=preview)
    img_widget = w.Image(value=open(preview, "rb").read())

    result.set_content([dwn, img_widget])
    
    #toggle the loading button
    widget.toggle_loading()
    
    return 

export_btn.on_event('click', partial(
    start_process,
    output=export_output,
    result = export_results
))
    

In [ ]:
export_tile

In [ ]:
export_results